In [2]:
import importlib
import vrt
from osgeo import gdal
import pprint
import json
import fmask
import numpy as np
import matplotlib.pyplot as plt
from variables import DATA_PATH
import itertools
from collections import Counter
importlib.reload(vrt)
importlib.reload(fmask)

# 0 => clear land pixel
# 1 => clear water pixel
# 2 => cloud shadow
# 3 => snow
# 4 => cloud

<module 'fmask' from '/Users/thomashebrard/thesis/code/fmask.py'>

In [3]:
import os

directory = f"{DATA_PATH}/s2/fmask/"  # replace with the path to your directory
fmask_paths = [
    f"{DATA_PATH}/s2/fmask/" +
    folder_name +
    "/" +
    folder_name +
    "_fmask.tif"
    for folder_name in os.listdir(directory)
]

In [4]:
MAP_CLOUD = {
    0: "clear_land",
    1: "cloud_water", 
    2: "cloud_shadow",
    3: "snow",
    4: "cloud",
    255: "no_observation"
}

In [ ]:
json_data = {}

for fmask_path in fmask_paths:
    vrt_path = "vrt/" + fmask_path.split("/")[-1].split(".")[0] + ".vrt"
    try:
        vrt.tif_to_vrt(fmask_path, vrt_path)
        gdal_data = gdal.Open(vrt_path)
        data = gdal_data.ReadAsArray()
    except:
        print("File not exist here: ", fmask_path)
        vrt.tif_to_vrt("/".join(fmask_path.split("/")[:-1]) + "/fmask.tif", vrt_path)
        gdal_data = gdal.Open(vrt_path)
        data = gdal_data.ReadAsArray()
    
    shape = data.size
    
    count = dict(Counter(itertools.chain(*data)))
    map = {MAP_CLOUD[k]: round((v/shape)*100, 2) for k, v in count.items()}
    
    json_data[fmask_path.split("/")[-1]] = dict(sorted(map.items(), key=lambda x: -x[1]))

with open("fmask_stats.json", "w") as f:
    json.dump(json_data, f)